## Label Drift

In [1]:
import numpy as np
import pandas as pd

from deepchecks.tabular import Dataset
from deepchecks.tabular.checks import LabelDrift

### Carregamento dos dados de 2017 e 2021

In [5]:
enade_treino = pd.read_csv("../concept_drift/tabela_final_2017_treinamento.csv")
enade_teste = pd.read_csv("../concept_drift/tabela_final_2021_treinamento.csv")

In [6]:
train_dataset = Dataset(enade_treino, label='Nota_Conceito_Faixa', cat_features=[])
test_dataset = Dataset(enade_teste, label='Nota_Conceito_Faixa', cat_features=[])

### Uso do método LabelDrift da biblioteca

In [7]:
check = LabelDrift()
result = check.run(train_dataset=train_dataset, test_dataset=test_dataset)

In [8]:
result

## Feature Drift

In [2]:
from deepchecks.tabular.checks import FeatureDrift
from sklearn.svm import SVC

### Carregamento dos dados de 2017 e 2021

In [3]:
enade_treino = pd.read_csv("../concept_drift/tabela_final_2017_treinamento.csv")

numero_caracteristicas = enade_treino.shape[1] - 1
X_enade_treino = enade_treino.iloc[:, 0:numero_caracteristicas]
y_enade_treino = enade_treino.iloc[:, -1]

enade_teste = pd.read_csv("../concept_drift/tabela_final_2021_treinamento.csv")

In [4]:
train_dataset = Dataset(enade_treino, label='Nota_Conceito_Faixa', cat_features=[])
test_dataset = Dataset(enade_teste, label='Nota_Conceito_Faixa', cat_features=[])

### Uso do método FeatureDrift da biblioteca

#### Instanciação do modelo de preferência

In [9]:
model = SVC(probability=True)
model.fit(X_enade_treino, y_enade_treino)

SVC(probability=True)

#### Uso do método

In [10]:
check = FeatureDrift()
result = check.run(train_dataset=train_dataset, test_dataset=test_dataset, model=model, feature_importance_force_permutation=True)

deepchecks - WARNING - Features importance was not calculated:
Skipping permutation importance calculation: calculation was projected to finish in 2443 seconds, but timeout was configured to 120 seconds


In [ ]:
result.value

In [12]:
result.show()

## Multivariate Drift

In [13]:
from deepchecks.tabular.checks import MultivariateDrift

In [14]:
enade_treino = pd.read_csv("../concept_drift/tabela_final_2017_treinamento.csv")

numero_caracteristicas = enade_treino.shape[1] - 1
X_enade_treino = enade_treino.iloc[:, 0:numero_caracteristicas]
y_enade_treino = enade_treino.iloc[:, -1]

enade_teste = pd.read_csv("../concept_drift/tabela_final_2021_treinamento.csv")

In [15]:
train_dataset = Dataset(enade_treino, label='Nota_Conceito_Faixa', cat_features=[])
test_dataset = Dataset(enade_teste, label='Nota_Conceito_Faixa', cat_features=[])

In [16]:
check = MultivariateDrift()
result = check.run(train_dataset=train_dataset, test_dataset=test_dataset)

In [ ]:
result.value

In [17]:
result.show()